In [1]:
import math
import numpy as np
import scipy.special as spf
import vegas # numeric integration
import gvar # gaussian variables; for vegas
import time
import quaternionic # For rotations
# import spherical #For Wigner D matrix
# import csv # file IO for projectFnlm
# import os.path
import h5py # database format for mathcalI arrays
import importlib
import sys
import matplotlib.pyplot as plt


sys.path.append('../')

import vsdm
from vsdm.units import *
# from vsdm.utilities import *
vsdm.__version__

'0.1.0'

## Form factor (box)

In [2]:
def fj2(nj, qLj):
    if qLj==0:
        if nj==1:
            return 1
        else:
            return 0        
    qlp = np.abs(qLj)/np.pi
    # mathsinc(x) = np.sinc(x/pi)
    s_minus = np.sinc(0.5*(qlp - nj + 1))/(1 + (nj-1)/qlp)
    s_plus = np.sinc(0.5*(qlp - nj - 1))/(1 + (nj+1)/qlp)
    return (s_minus - s_plus)**2

### MOMENTUM DISTRIBUTION EXAMPLES

# Long thin box limit: assuming that Lz > Lx,Ly, 
# so the lowest excited states are nz=2, nz=3, with nx=ny=1. 

def fs2_nz(Lvec, nz, q_xyz):
    # q: the DM particle velocity (cartesian, lab frame)
    # L: the dimensions of the box
    # nz = 2, 3, 4... The final state. (n=1 defined as ground state)
    # fs2 is dimensionless
    # note: np.sinc(x/pi) = sin(x) / (x). Included in defs. of qL below
    (Lx, Ly, Lz) = Lvec
    (qx, qy, qz) = q_xyz
    qLx = Lx*qx
    qLy = Ly*qy
    qLz = Lz*qz
#     qL = qLx + qLy + qLz
    fx2 = fj2(1, qLx)
    fy2 = fj2(1, qLy)
    fz2 = fj2(nz, qLz)
    return fx2*fy2*fz2

def DeltaE_nz(nz, Lz):
    # for nx=ny=1 final states, in units of [q**2]/mElec
    return 0.5*math.pi**2 / mElec * (nz**2 - 1)/Lz**2

def fs2_m1(qDMsph):
    # Taking Lz = 10*a0 for all examples, with DeltaE = 4.0285 eV for nz=2.
    Lx = 4/qBohr
    Ly = 4/qBohr
    Lz = 10/qBohr
    Lvec = (Lx, Ly, Lz)
    (q, theta, phi) = qDMsph
    q_xyz = (q * np.sin(theta) * np.cos(phi), 
              q * np.sin(theta) * np.sin(phi), 
              q * np.cos(theta))
    return fs2_nz(Lvec, 2, q_xyz)

def fs2_m2(qDMsph):
    # Taking Lz = 10*a0 for all examples, with DeltaE = 4.0285 eV for nz=2.
    Lx = 8/qBohr
    Ly = 8/qBohr
    Lz = 10/qBohr
    Lvec = (Lx, Ly, Lz)
    (q, theta, phi) = qDMsph
    q_xyz = (q * np.sin(theta) * np.cos(phi), 
              q * np.sin(theta) * np.sin(phi), 
              q * np.cos(theta))
    return fs2_nz(Lvec, 2, q_xyz)

def fs2_m3(qDMsph):
    # Taking Lz = 10*a0 for all examples, with DeltaE = 4.0285 eV for nz=2.
    Lx = 4/qBohr
    Ly = 7/qBohr
    Lz = 10/qBohr
    Lvec = (Lx, Ly, Lz)
    (q, theta, phi) = qDMsph
    q_xyz = (q * np.sin(theta) * np.cos(phi), 
              q * np.sin(theta) * np.sin(phi), 
              q * np.cos(theta))
    return fs2_nz(Lvec, 2, q_xyz)


#

In [4]:
QMAX = 10*qBohr # Global value for q0=qMax for wavelets

def fs2_model4(qSph):
    (q, theta, phi) = qSph
    qx = q*math.sin(theta) * math.cos(phi)
    qy = q*math.sin(theta) * math.sin(phi)
    qz = q*math.cos(theta)
    return fs2_nz((4/qBohr, 7/qBohr, 10/qBohr), 2, (qx,qy,qz))
fs2_model4.is_gaussian = False
fs2_model4.z_even = True
fs2_model4.phi_even = True
fs2_model4.phi_cyclic = 2


bdict = dict(u0=QMAX, type='wavelet', uMax=QMAX)
vegas_params = dict(neval=1e5, nitn_init=5, 
                    neval_init=1e3, nitn=10, verbose=True, 
                    weight_by_vol=1, neval_min=1e3)
print('{:.4G} keV per bin'.format(QMAX/128/keV))

nMax = 95
nMin = 0
lMin = 0
lMax = 36
nlmlist_3 = vsdm.makeNLMlist(nMax, lMax, nMin=nMin, lMin=lMin,
                             phi_even=True, mSymmetry=2, lSymmetry=2)

csvname = 'out/fs4_1e5.csv'

0.2913 keV per bin


In [ ]:
"""Initial numeric calculation."""
wave1 = vsdm.ProjectFnlm(bdict, fs2_model4, vegas_params, 
                         nlmlist=nlmlist_3,  
                         f_type='fs2', 
                         csvsave_name=csvname)
print(wave1.t_eval)


In [ ]:
"""Improvement on most important coeffs..."""
# # Read Fnlm from saved csv file...
wave_csv = vsdm.ProjectFnlm(bdict, fs2_model4, vegas_params, 
                            nlmlist=None, f_type='fs2')
wave_csv.importFnlm_csv(csvname)
# get the energy from fnlm...
energy = wave_csv.f2nlm_energy()
# get fractional power:
powerNLM = wave_csv.getNLMpower()
power_fractional = {key:value/energy for key,value in powerNLM.items()}

neval_BIG = 1e6
prec_goal_energy = 1e-4
vegas_update = dict(neval=neval_BIG, nitn=10, 
                    neval_init=1e3, nitn_init=5, verbose=True, 
                    weight_by_vol=False, neval_min=1e3)

# new integrals:
for nlm,frac_power in power_fractional.items():
    if frac_power > prec_goal_energy:
        neval_importance = int(frac_power.mean*neval_BIG)
        if neval_importance > vegas_update['neval_min']:
            vegas_update['neval'] = neval_importance
        wave_csv.updateFnlm(nlm, vegas_update, 
                            csvsave_name=csvname) 



## Velocity distributions (Gaussian)

In [5]:
def gaussian_stream_sum(ci, vWsph_i, sigma_i):
    # Arguments: lists of amplitudes gi, dispersions v0_i, and 
    #     lab-frame DM wind vectors vWsph_i, in spherical coordinates
    gvec_list = []
    for i in range(len(gi)):
        gaus = (ci[i], vWsph_i[i], sigma_i[i])
        gvec_list += [gaus]
    return gvec_list 

# Define a function to convert GaussianF(gX) into GaussianF(tilde_gX),
# for dimensionless function tilde_gX = u0**3 * gX,
# where u0 is the vsdm.Basis.u0 scale factor
def gX_to_tgX(gauF, u0):
    tgauF_vecs = gauF.rescaleGaussianF(u0**3)
    return vsdm.GaussianFnlm(gauF.basis, tgauF_vecs)


In [6]:
# Model 4: a bunch of streams, not symmetric. 
# Including the halo component without vEsc.
v0_main = 220*km_s
v0_a = 70*km_s
v0_b = 50*km_s
v0_c = 25*km_s
# vX_main = (230*km_s, np.pi, 0)
# vX_a = (113.13*km_s, 0.75*np.pi, 0) 
# vX_b = (315.28*km_s, 2.067, 4.265)
# vX_c = (1.5*256.71*km_s, 2.912, 0.540)
vX_main = vsdm.cart_to_sph((0, 0, -230*km_s))
vX_a = vsdm.cart_to_sph((80*km_s, 0, -80*km_s))
vX_b = vsdm.cart_to_sph((-120*km_s, -250*km_s, -150*km_s))
vX_c = vsdm.cart_to_sph((50*km_s, 30*km_s, -400*km_s))
sigma_i = [v0_main, v0_a, v0_b, v0_c]
vWsph_i = [vX_main, vX_a, vX_b, vX_c]
gi = [0.4, 0.3, 0.2, 0.1]
gvec_list_4 = gaussian_stream_sum(gi, vWsph_i, sigma_i)

VMAX = 960.*km_s # Global value for v0=vMax for wavelets
bdict = dict(u0=VMAX, type='wavelet', uMax=VMAX)
gXmodel_4 = vsdm.GaussianFnlm(bdict, gvec_list_4)
gtilde_4 = gX_to_tgX(gXmodel_4, VMAX)
gvec_tilde_4 = gtilde_4.gvec_list

In [6]:
gvegas_params = dict(neval=1e4, nitn=7, nitn_init=3, 
                     weight_by_vol=True, neval_min=3e2,
                     neval_init=3e2, verbose=False)

doNumericEvaluation = True
modelname = 'gX_model4'

csvname = 'demo/' + modelname + '.csv'

if doNumericEvaluation:
    """Numeric evaluation..."""
    nMax = 192
    nMin = 0
    ellMax = 36
    nlmlist_4 = [(n,ell,m) for ell in range(ellMax+1) 
                 for m in range(-ell, ell+1) 
                 for n in range(nMin, nMax)]

    wave4 = vsdm.ProjectFnlm(bdict, gtilde_4, gvegas_params, 
                             nlmlist=nlmlist_4, f_type='gX', 
                             csvsave_name=csvname, use_gvar=True)
#     wave4.writeFnlm_csv(csvname)
    wave4.writeFnlm('out/draftplots.hdf5', modelname, 
                    writeGnli=True)
else:
    """Read from hdf5..."""
    wave4 = vsdm.ProjectFnlm(bdict, gtilde_4, gvegas_params, 
                              nlmlist=None, f_type='gX')
    wave4.importFnlm_csv(csvname)
    print('nCoeffs = {}'.format(len(wave4.getNLMlist())))
    print('is_gaussian: ', wave4.is_gaussian)

print(wave4.t_eval)


1574.7977797985077


In [ ]:
"""Improvements..."""
# gvegas_better = dict(neval=1e5, nitn=7, nitn_init=3, 
#                      neval_init=1e3, verbose=True)
# for n in range(192, 256):
#     nlm = (n, ell, m)
#     wave4.updateFnlm(nlm, gvegas_params)

# print(wave4.t_eval)
# print(wave4.f_nlm[(0,0,0)])
# print(wave4.gU((0,0,0)))
# print(wave4.nlmFu((0,0,0)))


In [ ]:
"""Optionally, save the more precise values to CSV."""
# wave4.writeFnlm_csv('demo/gX_model4_precise.csv')
